In [1]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora
from tqdm import tqdm_notebook
from pprint import pprint

# 사용자간의 유사성을 평가하기 위한 measure를 사용하기 위해 불러오기
from gensim.matutils import hellinger
from gensim import matutils

import pandas as pd
import os

In [2]:
NUM_TOPICS = 4

if not os.path.exists("kakao(ATM)_model"):
    model = AuthorTopicModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary.id2token, \
                author2doc=author2doc, passes=10)
    model.save('kakao(ATM)_model')
else:
    model = AuthorTopicModel.load("kakao(ATM)_model")

In [3]:
# 토픽별 분포 확인하기
model.show_topic(0)

[('닥쳐', 0.00913352474795045),
 ('오우', 0.006077150842656581),
 ('핡', 0.005814291204809277),
 ('넌', 0.004275330190804228),
 ('그게', 0.003936329056179983),
 ('찍', 0.003518902685383488),
 ('됨', 0.0033362068393125244),
 ('일단', 0.0031497496609154147),
 ('가서', 0.0030418551056656984),
 ('올', 0.002905564005796451)]

In [4]:
# 토픽 라벨 지정.
topic_labels = ["Topic0", "Topic1", "Topic2", "Topic3"]

In [5]:
# 토픽별로 topN 단어 확인하기.
for topic in model.show_topics(NUM_TOPICS):
    print('Label: ' + topic_labels[topic[0]])
    words = ''
    for word, prob in model.show_topic(topic[0], topn=20):
        words += word + ' '
    print('Words: ' + words)
    print()

Label: Topic0
Words: 닥쳐 오우 핡 넌 그게 찍 됨 일단 가서 올 뭐야 야 집 이게 신촌 둘다 기모찌 팬더 도착 이건 

Label: Topic1
Words: 나는 다 이제 그냥 당연 역시 뭐 같이 이미 가능 좋음 왜 진짜 튜브 그래도 솬 곧 존나 있음 프로도신 

Label: Topic2
Words: 예아 괜찮 갑자기 이꾸 하앙 둘이 보면 있는데 뭐임마 컴 미친 같음 개꿀 연구실 밥임 그러게 항상 치킨 솔직히 아닌데 

Label: Topic3
Words: 굿 아 난 프로도 근데 핳 허허 오늘 나도 내가 나 오 라이언 그럼 굳 프로도이 지금 어피치 핳핳 좀 



In [6]:
# 사용자별로 토픽 분포 확인하기.
def show_author(name):
    print('\n%s' % name)
    print('Docs:', model.author2doc[name])
    print('Topics:')
    pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [7]:
# "네오"의 토픽 분포 확인.
show_author('네오')


네오
Docs: [273, 274, 276, 277, 342, 374, 396, 3799, 3800, 3802, 3806, 3808, 3814, 3816, 3818, 3820, 3823, 3824, 3826, 3828, 3831, 3834, 3835, 3838, 3841, 3863, 3864, 3865, 3866, 3870, 3872, 3877, 3885, 3888, 3889, 3904, 3905, 3907, 3912, 3916, 3920, 3926, 3944, 3949, 3951, 3952, 3953, 3958, 3962, 4433, 4434, 4442, 4444, 4451, 4452, 4453, 4455, 4459, 4460, 4466, 4467, 4469, 4473, 4476, 4477, 4479, 4487, 4489, 4495, 4497, 4499, 4505, 4506, 4508, 4510, 4513, 4515, 4516, 4517, 4519, 4521, 4523, 4527, 4530, 4533, 4535, 4536, 4542, 4547, 4551, 4556, 4558, 15826, 32378, 32379, 32383, 32385, 32386, 32388, 32405, 32408, 32412, 32413, 32416, 32419, 42387, 42390, 42391, 42394, 42399, 42410, 42489, 42492, 42493, 42494, 42496, 45428, 45431, 45438, 45439, 45457, 45458, 48980, 48981, 48982, 48983, 48989, 49907, 49909, 49912, 49913, 49915, 49917, 49919, 49921, 49938, 49941, 49943, 49945, 49946, 49949, 84471, 84472, 84477, 84478, 84482, 84485, 84489, 84491, 85677, 93507, 93508, 93510, 93511, 107221, 10

In [12]:
[model.get_author_topics(author) for author in model.id2author.values()]

[[(0, 0.14798956342038022),
  (1, 0.30637663906281004),
  (2, 0.15846342670808514),
  (3, 0.38717037080872463)],
 [(0, 0.0880999332585824),
  (1, 0.493662980893337),
  (2, 0.14462030211383983),
  (3, 0.2736167837342408)],
 [(0, 0.13377666097787727),
  (1, 0.21083885589015972),
  (2, 0.0971719396849559),
  (3, 0.5582125434470071)],
 [(0, 0.11025390157874199),
  (1, 0.4469268384813222),
  (2, 0.07415173994093803),
  (3, 0.3686675199989978)],
 [(0, 0.04065800921486131),
  (1, 0.20711683329874),
  (2, 0.1468495200964532),
  (3, 0.6053756373899456)],
 [(0, 0.3438993754743885),
  (1, 0.25765725089249314),
  (2, 0.11776001705181706),
  (3, 0.2806833565813014)]]

In [8]:
# Hellinger Distance를 이용하여 비슷한 토픽을 가진 사용자를 추정하는 함수.


# author-topic 분포 만들기
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]
 
def similarity(vec1, vec2):
    # vec1, vec2 사이의 hellinger similarity 구하기.
    dist = hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim
 
def get_sims(vec):
    # 각 사용자들 사이의 similarity pair 구하기.
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims
 
def get_table(name, top_n=10, smallest_author=1):
    """
    주어진 사용자에 대해서 topN 사람만큼 유사도를 정렬해서 table을 출력하는 함수입니다.
    """
    
    # 유사도 측정하기
    sims = get_sims(model.get_author_topics(name))
 
    # 저자별 정보 정렬하기
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            
    # 사용자 패턴 분석 결과를 Dataframe으로 만들기
    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [9]:
# 사용자별 대화 패턴 검증
get_table('무지')

,Author,Score,Size
2,무지,1.000000,48320
0,네오,0.887220,324
4,튜브,0.886143,42358
3,어피치,0.846163,12741
5,프로도,0.817804,20514
1,라이언,0.803962,39311
